## 이 노드의 루브릭

#### 1. BERT pretrained model을 활용한 KorQuAD 모델이 정상적으로 학습되었다.    
: KorQuAD 모델의 validation accuracy가 안정적으로 증가하였다.     

#### 2. KorQuAD Inference 결과가 원래의 정답과 비교하여 유사하게 나오는 것을 확인하였다.      
: 평가셋에 대해 모델 추론 결과와 실제 정답의 유사성이 확인되었다.    

#### 3. Pretrained Model 활용이 효과적임을 실험을 통해 확인하였다.         
: pretrained model을 사용하지 않았을 때 대비, 학습경과의 차이를 시각화를 통해 확인하였다.

## 목차

### 1. 데이터 로드 및 데이터 전처리            

### 2. Not-pretrained 학습 결과 확인하기

### 3. Pretrained model 로딩하기     

### 4. pretrained model finetune 하기      

### 5. Inference 수행하기       

### 6. 학습 경과 시각화 비교분석

---

## 1. 데이터 로드 및 데이터 전처리

In [2]:
#필요한 라이브러리 임포트
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

import os
import re

import numpy as np
import pandas as pd

import pickle  
import random

import collections
import json
from datetime import datetime

import sentencepiece as spm
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

random_seed = 1234
#랜덤 시드 배정
random.seed(random_seed)

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

print('임포트 완료')

임포트 완료


- 오류가 생겨서 pip freeze로 버전을 확인하였다.                     
- tensorflow와 tensorflow -addons의 버전은 각각 2.2.0, 0.12.1          
- pip install tensorflow-addons==0.11.2로 다운그레이드                
- [이 곳에서 tensorflow와 tensorflow-addons의 버전을 참고](https://stackoverflow.com/questions/65464463/importerror-cannot-import-name-keras-tensor-from-tensorflow-python-keras-eng)              

### 데이터 로드 및 내용 확인

In [10]:
def print_json_tree(data, indent=""):
    for key, value in data.items():
        if type(value) == list:     
            print(f'{indent}- {key}: [{len(value)}]')
            print_json_tree(value[0], indent + "  ")
        else:
            print(f'{indent}- {key}: {value}')
            
print('json data 프린트 함수 세팅')

json data 프린트 함수 세팅


In [4]:
#데이터 패스 입력
data_dir = os.getenv('HOME') + '/SUBMIT_MISSION_GIT/ex19_BERT/Data'
model_dir =  os.getenv('HOME') + '/SUBMIT_MISSION_GIT/ex19_BERT/Model'

In [11]:
#훈련 데이터 확인
train_json_path = data_dir + '/KorQuAD_v1.0_train.json'

with open(train_json_path) as f:
    train_json = json.load(f)
    print_json_tree(train_json)

- version: KorQuAD_v1.0_train
- data: [1420]
  - paragraphs: [3]
    - qas: [8]
      - answers: [1]
        - text: 교향곡
        - answer_start: 54
      - id: 6566495-0-0
      - question: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
    - context: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.
  - title: 파우스트_서곡


In [12]:
#검증 데이터 확인
vali_json_path = data_dir + '/KorQuAD_v1.0_dev.json'

with open(vali_json_path) as f:
    vali_json = json.load(f)
    print_json_tree(vali_json)

- version: KorQuAD_v1.0_dev
- data: [140]
  - paragraphs: [2]
    - qas: [7]
      - answers: [1]
        - text: 1989년 2월 15일
        - answer_start: 0
      - id: 6548850-0-0
      - question: 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
    - context: 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.
  - title: 임종석


In [13]:
#Json의 실제 데이터를 json.dumps()로 확인해보기!

print(json.dumps(train_json["data"][0], indent=2, ensure_ascii=False))

{
  "paragraphs": [
    {
      "qas": [
        {
          "answers": [
            {
              "text": "교향곡",
              "answer_start": 54
            }
          ],
          "id": "6566495-0-0",
          "question": "바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?"
        },
        {
          "answers": [
            {
              "text": "1악장",
              "answer_start": 421
            }
          ],
          "id": "6566495-0-1",
          "question": "바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?"
        },
        {
          "answers": [
            {
              "text": "베토벤의 교향곡 9번",
              "answer_start": 194
            }
          ],
          "id": "6566495-0-2",
          "question": "바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?"
        },
        {
          "answers": [
            {
              "text": "파우스트",
              "answer_start": 15
            }
          ],
          "id": "6566518-0-0",
          "question": "1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?"
        },
        {
    

### 데이터 전처리        

- ### 1. 띄어쓰기 단위 정보 관리

In [14]:
def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [33]:
#공백을 기준으로 토큰화를 수행하는 함수 정의

def _tokenize_whitespace(string):
    word_tokens=[]
    char_to_word=[]
    prev_is_whitespace=True
    
    for c in string:
        if is_whitespace(c):
            prev_is_whitespace = True
            
        else:
            #이전이 공백이라면 새로운 워드 토큰으로 분류
            if prev_is_whitespace:
                word_tokens.append(c)
            else:
            #이전이 공백이 아니라면 같은 워드 토큰으로 분류
                word_tokens[-1] += c
                
            #분류를 끝내고 이전공백=False
            prev_is_whitespace = False
            
        #전체 char_to_word에 append
        char_to_word.append(len(word_tokens)-1)
        
    return word_tokens, char_to_word

- ## 2. 형태소 단위 토큰화

- 한국어의 경우, 동사의 활용형이 지나치게 많기 때문에 같은 형태소를 공유하는 다양한 어미를 모두 다른 워드벡터로 분류하면 차원 수가 엄청나게 늘어나게 된다.      
- 이를 방지하기 위해 __Subword Segmentation__ 을 활용한다.     
- SentencePiece 모델을 활용한다.

In [30]:
#Vocab 모델 로드
vocab = spm.SentencePieceProcessor()
vocab.load(f"{model_dir}/ko_32000.model")

True

In [34]:
#형태소 단위 토큰화 함수 세팅
def _tokenize_vocab(vocab, context_words):
    #각 단위의 길이
    word_to_token=[]
    #형태소 단위로 분리된 단어
    context_tokens=[]
    
    for (i, word) in enumerat다e(context_words):
        word_to_token.append(len(context_tokens))
        
        #형태소 단위 토큰을 분리의
        tokens = vocab.encode_as_pieces(word)
        
        for token in tokens:
            context_tokens.append(token)
            
    return context_tokens, word_to_token

__함수세팅 검증__

In [41]:
string1 = '1839년 파우스트을 읽었다.'

In [42]:
_tokenize_whitespace(string1)

(['1839년', '파우스트을', '읽었다.'], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2])

- ## 3. Improve Span     


- KorQuAD에서 질문(Question), 지문(Context), 정답(Answer)을 찾아내 본다.

In [44]:
#json의 파일 구조를 기반으로 질문, 지문, 정답을 추출해본다.

context = train_json['data'][0]['paragraphs'][0]['context']
question = train_json['data'][0]['paragraphs'][0]['qas'][0]['question']
answer_text = train_json['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
answer_start = train_json['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']
answer_end = answer_start + len(answer_text) -1

In [47]:
#프린트로 검증
print('[context] ', context)
print('[question] ', question)
print('[answer] ', answer_text)
print('[answer_start] index: ', answer_start, 'character: ', context[answer_start])
print('[answer_end]index: ', answer_end, 'character: ', context[answer_end])

[context]  1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.
[question]  바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
[answer]  교향곡
[answer_start] index:  54 character:  교
[answer_end]index:  56 character:  곡


#### 위의 1.띄어쓰기 단위 토큰화, 2.형태소 단위 토큰화를 콘텍스트에 적용해본다.

In [48]:
word_tokens, char_to_word = _tokenize_whitespace(context)

print('콘텍스트를 띄어쓰기 단위 토큰화', word_tokens[:20])

char_to_word[:20], context[:20]

콘텍스트를 띄어쓰기 단위 토큰화 ['1839년', '바그너는', '괴테의', '파우스트을', '처음', '읽고', '그', '내용에', '마음이', '끌려', '이를', '소재로', '해서', '하나의', '교향곡을', '쓰려는', '뜻을', '갖는다.', '이', '시기']


([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3],
 '1839년 바그너는 괴테의 파우스트을')

In [50]:
context_tokens, word_to_token = _tokenize_vocab(vocab, word_tokens)

for i in range(min(20, len(word_to_token)-1)):
    print(word_to_token[i], context_tokens[word_to_token[i] :word_to_token[i+1]])

0 ['▁1839', '년']
2 ['▁바그너', '는']
4 ['▁괴테', '의']
6 ['▁', '파우스트', '을']
9 ['▁처음']
10 ['▁읽고']
11 ['▁그']
12 ['▁내용에']
13 ['▁마음이']
14 ['▁끌려']
15 ['▁이를']
16 ['▁소재로']
17 ['▁해서']
18 ['▁하나의']
19 ['▁교향곡', '을']
21 ['▁쓰', '려는']
23 ['▁뜻을']
24 ['▁갖는다', '.']
26 ['▁이']
27 ['▁시기']


> 여기서, 정답 글자의 시작 인덱스와 끝 인덱스를 띄어쓰기 단위로 변환하면 어떻게 될까?

In [53]:
#answer_start와 answer_end로 부터 word_start와 word_end를 구한다.

word_start = char_to_word[answer_start]
word_end = char_to_word[answer_end]

print('word_start/', 'word_end/', 'answer_text/', 'word_tokens[word_start:word_end+1]')
word_start, word_end, answer_text, word_tokens[word_start:word_end+1]

word_start/ word_end/ answer_text/ word_tokens[word_start:word_end+1]


(14, 14, '교향곡', ['교향곡을'])

> 답은 14번째에 있음을 인덱스로는 찾았으나, 띄어쓰기로 찾았을 때 '교향곡을' 로 찾게 된다.        
> __이를 수정하기 위해 word_start~word_end 까지를 형태소 단위로 찾아보자__

In [54]:
token_start = word_to_token[word_start]

if word_end < len(word_to_token)-1:
    token_end = word_to_token[word_end+1]-1
else:
    token_end = len(context_tokens)-1
    
print('token_start/', 'token_end/', 'context_tokens[token_start:token_end+1]')
token_start, token_end, context_tokens[token_start:token_end+1]

token_start/ token_end/ context_tokens[token_start:token_end+1]


(19, 20, ['▁교향곡', '을'])

> - 정답만 쏙! 빼오기 위해 __answer_text를 형태소 단위로 토큰화__ 해둔다.

In [55]:
token_answer = " ".join(vocab.encode_as_pieces(answer_text))

token_answer

'▁교향곡'

### Context에서 answer의 위치를 subword상태에서 토큰화된 상태에서 찾는 함수를 세팅

In [56]:
def _improve_span(vocab, context_tokens, token_start, token_end, char_answer):
    
    #char_answer를 넣고 형태소 단위로 토큰화된 걔를 얻는다!
    token_answer = " ".join(vocab.encode_as_pieces(char_answer))
    
    for new_start in range(token_start, token_end+1):
        for new_end in range(token_end, new_start-1, -1):
            text_span = " ".join(context_tokens[new_start : (new_end)+1])
            
            #답안이 이미 형태소 단위 토큰화와 일치할 때
            if text_span == token_answer:
                return (new_start, new_end)
            
    return (token_start, token_end)

- ## 4.데이터셋 분리 

- train 데이터셋, validation 데이터셋을 분리하고     
- improve_span 함수를 통해 최종적으로 전처리한 후      
- 파일로 저장합니다.

In [67]:
def dump_korquad(vocab, json_data, out_file):
    #우선 쓰기전용으로 파일을 데려와
    with open(out_file, "w") as f:
        
        #이름은 title 컬럼으로 붙여주고.
        for data in tqdm(json_data["data"]):
            title = data['title']
        
            #패러그래프를 가져와서, 띄어쓰기 단위로 분절
            for paragraph in data["paragraphs"]:
                context = paragraph["context"]
                context_words, char_to_word = _tokenize_whitespace(context)
                
                for qa in paragraph["qas"]:
                    
                    #만약 정답의 길이가 1이라면(더 자르고 말고 할 게 없는 상태)
                    #빠져나가도록 assert
                    assert len(qa["answers"]) == 1
                    
                    qa_id = qa["id"]
                    question = qa["question"]
                    
                    answer_text = qa["answers"][0]["text"]
                    answer_start = qa["answers"][0]["answer_start"]
                    answer_end = answer_start + len(answer_text) - 1
                    
                    #만약 데이터 안의 정답과, 분리한 context 안의 정답이 같다면(정리할 게 없다면)
                    #빠져나가도록 assert 처리
                    assert answer_text == context[answer_start:answer_end + 1]
                    
                    #워드 스타트 = 정답 시작점
                    word_start = char_to_word[answer_start]
                    word_end = char_to_word[answer_end]
                    
                    word_answer = " ".join(context_words[word_start:word_end + 1])
                    char_answer = " ".join(answer_text.strip().split())
                    
                    #이미 char_answer이 word_answer안에 있어서 처리할 필요 없다면
                    #빠져나가도록 assert 처리
                    assert char_answer in word_answer
                    
                    context_tokens, word_to_token = _tokenize_vocab(vocab, context_words)
                    
                    token_start = word_to_token[word_start]
                    if word_end < len(word_to_token) - 1:
                        token_end = word_to_token[word_end + 1] - 1
                    else:
                        token_end = len(context_tokens) - 1

                    token_start, token_end = _improve_span(vocab, context_tokens, token_start, token_end, char_answer)

                    data = {"qa_id": qa_id, "title": title, "question": vocab.encode_as_pieces(question), "context": context_tokens, "answer": char_answer, "token_start": token_start, "token_end":token_end}
                    f.write(json.dumps(data, ensure_ascii=False))
                    f.write("\n")
                    
print('최종 전처리 및 파일 저장 함수 세팅')

최종 전처리 및 파일 저장 함수 세팅


In [68]:
dump_korquad(vocab, train_json, f"{data_dir}/korquad_train.json")
dump_korquad(vocab, vali_json, f"{data_dir}/korquad_dev.json")

  0%|          | 0/1420 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

In [69]:
#최종 전처리가 잘 되었는지 확인해보자구!
def print_file(filename, count=10):
    """
    파일 내용 출력
    :param filename: 파일 이름
    :param count: 출력 라인 수
    """
    with open(filename) as f:
        for i, line in enumerate(f):
            if count <= i:
                break
            print(line.strip())

print_file(f"{data_dir}/korquad_train.json")

{"qa_id": "6566495-0-0", "title": "파우스트_서곡", "question": ["▁바그너", "는", "▁괴테", "의", "▁", "파우스트", "를", "▁읽고", "▁무엇을", "▁쓰고", "자", "▁", "했", "는", "가", "?"], "context": ["▁1839", "년", "▁바그너", "는", "▁괴테", "의", "▁", "파우스트", "을", "▁처음", "▁읽고", "▁그", "▁내용에", "▁마음이", "▁끌려", "▁이를", "▁소재로", "▁해서", "▁하나의", "▁교향곡", "을", "▁쓰", "려는", "▁뜻을", "▁갖는다", ".", "▁이", "▁시기", "▁바그너", "는", "▁1838", "년에", "▁빛", "▁독", "촉", "으로", "▁산", "전", "수", "전을", "▁다", "▁", "걲", "은", "▁상황이", "라", "▁좌절", "과", "▁실망", "에", "▁가득", "했으며", "▁메", "피스", "토", "펠", "레스", "를", "▁만나는", "▁", "파우스트", "의", "▁심", "경에", "▁공감", "했다고", "▁한다", ".", "▁또한", "▁파리에서", "▁아브", "네", "크의", "▁지휘", "로", "▁파리", "▁음악원", "▁관현악단", "이", "▁연주하는", "▁베토벤", "의", "▁교향곡", "▁9", "번을", "▁듣고", "▁깊은", "▁감", "명을", "▁받았는데", ",", "▁이것이", "▁이듬해", "▁1", "월에", "▁", "파우스트", "의", "▁서", "곡으로", "▁쓰여진", "▁이", "▁작품에", "▁조금", "이라도", "▁영향을", "▁끼", "쳤", "으리라", "는", "▁것은", "▁의심", "할", "▁여지가", "▁없다", ".", "▁여기", "의", "▁라", "단", "조", "▁조성", "의", "▁경우에도", "▁그의", "▁전기", "에", "▁적혀", "▁있는", 

- ## 5. 데이터셋 로드

- 지금까지 만든 데이터셋을 메모리에 로드한다.

In [70]:
train_json = os.path.join(data_dir, "korquad_train.json")
vali_json = os.path.join(data_dir, "korquad_dev.json")